# Twitter-Sentimental AnalysiswithFlair

## 1. Import and Data Load
- import and flair data_load

### 1.1 필요한 라이브러리 import

#### 1.1.1 Colab을 활용하기 위해서 torch, torchvision, matplotlib, nltk, textblob, vader, flair, twython 설치 

In [2]:
# High Runtime Check
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
# GPU Check
import torch

if torch.cuda.is_available():
  device_count = torch.cuda.device_count()
  print("device_count: {}".format(device_count))
  for device_num in range(device_count):
    print("device {} capability {}".format(device_num,torch.cuda.get_device_capability(device_num)))
    print("device {} name {}".format(device_num, torch.cuda.get_device_name(device_num)))
else:
  print("no cuda device")

device = torch.device('cuda')

device_count: 1
device 0 capability (6, 0)
device 0 name Tesla P100-PCIE-16GB


In [4]:
!pip install sacremoses
!pip install -U textblob
!python -m textblob.download_corpora
!pip install --upgrade pandas
!pip install --upgrade numpy
!pip install tweepy
!pip install torch 
!pip install torchvision 
!pip install matplotlib
!pip install nltk
!pip install vader
!pip install flair
!pip install twython
!pip install tqdm

     |████████████████████████████████| 880 kB 5.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=4b4075aca8bfebc13d70e9f2476cb0fe099a6ff19af59a845a3627bdbce7e52e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
     |████████████████████████████████| 636 kB 5.1 MB/s 
  Attempting uninstall: textblob
    Found existing installation: textblob 0.15.3
    Uninstalling textblob-0.15.3:
      Successfully uninstalled textblob-0.15.3
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]  

In [5]:
# nltk 활용을 위한 불용어 다운로드
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#### 1.1.2 구글 드라이브 연결

In [6]:
##구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
cd /content/drive/MyDrive/Capstone

/content/drive/MyDrive/Capstone


#### 1.1.3 필요한 library import

In [8]:
import sys
sys.path.append('/content/drive/MyDrive/Capstone')

In [16]:
import pandas as pd
from sentimentalAnalysis import sentimental_analysis
import matplotlib.pyplot as plt
import tqdm

### 1.2 필요한 데이터 로드 

- 13개의 flair로 processsed된 데이터 활용

In [17]:
flair_data=[]
for i in range(18,31):
  flair_data.append(pd.read_csv('flair_processed_data_kaggle(btc_bitcoin)_train_data'+str(i),index_col=0))
flair_data[8]

,date,tweet,clean_text,flair
2500000,2022-03-16 20:00:11,@Bitcoin I WILL GIVE YOU AN EQUATION THAT WILL...,I WILL GIVE YOU AN EQUATION THAT WILL CHANGE T...,NaN
2500001,2022-03-16 20:00:11,"The price of\nSpaghetti alla Carbonara\nis 26,...",The price of Spaghetti alla Carbonara is 26 86...,NaN
2500002,2022-03-16 20:00:09,Trust is the reason we are ✊📱\nTrust is why we...,Trust is the reason we are Trust is why we bui...,NaN
2500003,2022-03-16 20:00:09,Are you looking for mindblowing APYs? 🤯🤑\n\nUs...,Are you looking for mindblowing APYs Use thank...,NaN
2500004,2022-03-16 20:00:09,I'm one of them and very happy 🥳👌🏼 #NativeDeFi...,I am one of them and very happy on is the Best...,NaN
...,...,...,...,...
2599995,2022-03-24 10:46:06,@HuobiGlobal @crypterium Ever-growing Ecosyste...,Ever growing Ecosystem Ever growing Numbers We...,NaN
2599996,2022-03-24 10:46:05,#DeFi #TeleFy #DEX #cryptocurrency #Ethereum #...,$ TELE,NaN
2599997,2022-03-24 10:46:00,Bank of England sketches out regulatory approa...,Bank of England sketches out regulatory approa...,NaN
2599998,2022-03-24 10:45:59,This zesty faucet from @_bitcoiner is making m...,This zesty faucet from @_bitcoiner is making m...,NaN


## 2. Flair 감성 분석

In [22]:
for i in range(0,13):
  sentimental_tweet = sentimental_analysis(flair_data[i])
  sentimental_tweet.sentimental_with_flair()
  sentimental_tweet.save_csv('flair_senti_train_data'+str(i+18))

2022-05-15 04:07:45,842 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


100%|██████████| 100000/100000 [13:03<00:00, 127.58it/s]


flair sentimental time :  789.7773098945618
2022-05-15 04:20:56,873 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


100%|██████████| 100000/100000 [13:02<00:00, 127.80it/s]


flair sentimental time :  786.4811899662018
2022-05-15 04:34:04,598 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


...

flair sentimental time :  776.9659559726715
2022-05-15 06:28:40,520 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


100%|██████████| 100000/100000 [12:51<00:00, 129.54it/s]


flair sentimental time :  774.4590382575989
2022-05-15 06:41:36,248 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


100%|██████████| 29118/29118 [03:43<00:00, 130.51it/s]


flair sentimental time :  225.59776401519775
